In [2]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text
import dotenv
import os

In [3]:
nba = pd.read_csv('ASA All NBA Raw Data.csv')

/var/folders/4w/k9sqqcbx4dxgpjtwgv_1m29h0000gq/T/ipykernel_10360/2306477363.py:1: DtypeWarning: Columns (68,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  nba = pd.read_csv('ASA All NBA Raw Data.csv')


In [4]:
dotenv.load_dotenv()
postgrespassword = os.getenv('postgrespassword')

In [5]:
pd.set_option('display.max_rows', 81)
nba.head(1).T

,0
game_id,202202170BRK
game_date,2022-02-17
OT,0
H_A,A
Team_Abbrev,WAS
Team_Score,117
Team_pace,94.5
Team_efg_pct,0.627
Team_tov_pct,13.5
Team_orb_pct,22.9


## Database Normalization
### First normal form:

1. **All tables must have a primary key**: In this table, `game_id` and `player_id` together are unique on every row, and so they form primary key.

2. **All the data must be atomic**: Inactives is non-atomic.

3. **No repeating groups problem**: We can't solve the non-atomicity problem by creating separate columns if this leads to arbitrary ordering language in the column names (for example, `Inactive1`, `Inactive2`, etc.) and if it leads to a lot of missing data (there would be an `Inactive7` which would be missing any time a team has less than 7 inactive players).

Our solution here is to cheat -- because we have `is_inactive` for each player, it makes `Inactives` redundant. So we can just delete `Inactives`.

If we did not have `is_inactive`, we would have to create a new table called `Inactive` with three columns: `game_id` and `inactive_player_id` that contains the player ID for each inactive player for each team in each game. Each inactive player in each game would get a new row. We wouldn't need to include team in this table because we could get the information about a player's team by joining this data to the player-game table. (A player plays for one team in one game, so if we know the game and player, we can lookup the player's team).

In [6]:
nba = nba.drop(['Inactives'], axis=1)

### Functional Dependence
Let X and Y be columns in a data table. Y is functionally dependent on X if each value of X has exactly one value of Y.

That's pretty abstract. So here are some guidelines that help me:

1. This use of "function" is the exact same as the concept of a function from algebra and pre-calculus. A correspondence f(x)=y is a function if each value of x has only one associated value of y.

2. X is either a primary key, or something that should be a primary key in another table.

For example, `game_date` (Y) is functionally dependent on `game_id` (X) because one `game_id` takes place on exactly one date.

### Second normal form:
In this table the primary key is a superkey consisting of two columns: `game_id` and `player_id`. 

2NF is violated if any columns are functionally dependent on part of the primary key but not the entire primary key. This can only happen if the primary key is a superkey.

Here there are three columns that depend on `game_id` but not on `player_id`: `game_date`, `OT`, and `season`. We solve 2NF by moving these columns to a new table.

There is also one column, `player`, that depends on `player_id` but not `game_id`. We create a new table here as well.

In [7]:
games = nba[['game_id', 'game_date', 'OT', 'season']].drop_duplicates()
players = nba[['player_id', 'player']].drop_duplicates()

### Third normal form:
3NF is violated if there are "transitive dependencies", that is, functional dependence between columns when neither column is part of the primary key.

In the main dataframe, `game_id` and `player_id` are part of the primary key. But many of the columns depend on `team_abbrev` as well as `game_id`. We pull these columns out and create a new table:

In [16]:
team_game = nba[['H_A', 'Team_Abbrev', 'Team_Score','Team_pace', 
                 'Team_efg_pct','Team_tov_pct','Team_orb_pct',
                 'Team_ft_rate','Team_off_rtg']].drop_duplicates()

In [19]:
player_game = nba.drop(['H_A', 'Team_Score','Team_pace', 
                        'Team_efg_pct','Team_tov_pct','Team_orb_pct',
                        'Team_ft_rate','Team_off_rtg', 'Opponent_Abbrev', 
                        'Opponent_Score','Opponent_pace','Opponent_efg_pct',
                        'Opponent_tov_pct','Opponent_orb_pct','Opponent_ft_rate',
                        'Opponent_off_rtg'], axis=1)

In [20]:
player_game.head(1).T

,0
game_id,202202170BRK
game_date,2022-02-17
OT,0
Team_Abbrev,WAS
player,Corey Kispert
player_id,kispeco01
starter,1
mp,32:30
fg,6
fga,9
